In [109]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [2]:
import requests
from requests.auth import HTTPBasicAuth

USERNAME = "232fd97ad7ef45d7a54e1f341f9e9ed8"
PASSWORD = "51A7F33F80CD483d8f12BEC230637840"

crt_file = "Diago.crt.txt"
pem_file = "Diago-private.pem"

resp = requests.get("https://api.non-prod.volkswagengroup.fr/tools/qa/leads/v2/dealers/export", params={"page": 2, "per_page": 10, "brand": "Volkswagen"}, auth=HTTPBasicAuth(USERNAME, PASSWORD), cert=(crt_file, pem_file))




In [57]:
resp = requests.get(
            "https://api.non-prod.volkswagengroup.fr/tools/qa/leads/v2/dealers/export", 
            params={"page": 0, "per_page": 10, "brand": "Volkswagen"},
            auth=HTTPBasicAuth(USERNAME, PASSWORD),
            cert=(crt_file, pem_file)
        )
resp.json()['data'][0]['address']

[{'use': 'billing',
  'line': [{'use': 'street', 'value': 'Z.I. LE PARQUET'}],
  'city': 'ST JEAN DE MAURIENNE',
  'postalCode': '73300',
  'country': 'France',
  'countryCode': 'FR',
  'longitude': '6.36427778',
  'latitude': '45.27122778'}]

In [67]:
resp = requests.get(
            "https://api.non-prod.volkswagengroup.fr/tools/qa/leads/v2/dealers/export", 
            params={"page": 1, "per_page": 10, "brand": "Volkswagen"},
            auth=HTTPBasicAuth(USERNAME, PASSWORD),
            cert=(crt_file, pem_file)
        )
resp.json()['data'][9]['address'][0]

{'use': 'billing',
 'line': [{'use': 'street', 'value': '107 Route DE CORBEIL'}],
 'city': 'STE GENEVIEVE BOIS CEDEX',
 'postalCode': '91704',
 'country': 'France',
 'countryCode': 'FR',
 'longitude': '2.33358056',
 'latitude': '48.64096389'}

In [123]:
resp = requests.get(
            "https://api.non-prod.volkswagengroup.fr/tools/qa/leads/v2/dealers/export", 
            params={"page": 57, "per_page": 10, "brand": "Volkswagen"},
            auth=HTTPBasicAuth(USERNAME, PASSWORD),
            cert=(crt_file, pem_file))
resp.json()['data'][2]['address'][0]

{'use': 'billing',
 'line': [{'use': 'street', 'value': '4 Rue DU CANAL Test Import'}],
 'city': 'LONGJUMEAU',
 'postalCode': '91160',
 'country': 'France',
 'countryCode': 'FR',
 'longitude': '2.63787222',
 'latitude': '48.56877222'}

In [97]:
resp = requests.get(
            "https://api.non-prod.volkswagengroup.fr/tools/qa/leads/v2/dealers/export", 
            params={"page": 57, "per_page": 10, "brand": "Volkswagen"},
            auth=HTTPBasicAuth(USERNAME, PASSWORD),
            cert=(crt_file, pem_file))
for i in range(10):
    try:
        conc = resp.json()['data'][i]['address'][0]
        print(conc['city'])
    except IndexError:
        print(f"Index out of range at: {i}")
        break
        

ERVY LE CHATEL
AVIGNON
LONGJUMEAU
Index out of range at: 3


In [101]:
# concessions_exploitables = []
# concessions_non_exploitables = []
# for page in range(1,57):
#     resp = requests.get(
#             "https://api.non-prod.volkswagengroup.fr/tools/qa/leads/v2/dealers/export", 
#             params={"page": page, "per_page": 10, "brand": "Volkswagen"},
#             auth=HTTPBasicAuth(USERNAME, PASSWORD),
#             cert=(crt_file, pem_file))
#     for i in range(10):
        
#         conc = resp.json()['data'][i]['address'][0]
#         if 'longitude' and 'latitude' in conc:
#             d= {k: conc[k] for k in ['city', 'postalCode', 'longitude', 'latitude']}
#             concessions_exploitables.append(d)
#         else:
#             d= {k: conc[k] for k in ['city', 'postalCode']}
#             concessions_non_exploitables.append(d)
# resp = requests.get(
#             "https://api.non-prod.volkswagengroup.fr/tools/qa/leads/v2/dealers/export", 
#             params={"page": 57, "per_page": 10, "brand": "Volkswagen"},
#             auth=HTTPBasicAuth(USERNAME, PASSWORD),
#             cert=(crt_file, pem_file))
# for i in range(3):
#     conc = resp.json()['data'][i]['address'][0]
#     if 'longitude' and 'latitude' in conc:
#         d= {k: conc[k] for k in ['city', 'postalCode', 'longitude', 'latitude']}
#         concessions_exploitables.append(d)
#     else:
#         d= {k: conc[k] for k in ['city', 'postalCode']}
#         concessions_non_exploitables.append(d)

# len(concessions_exploitables), len(concessions_non_exploitables)

(445, 118)

In [124]:
concessions_exploitables = []
concessions_non_exploitables = []

for page in range(1, 58):
    resp = requests.get(
        "https://api.non-prod.volkswagengroup.fr/tools/qa/leads/v2/dealers/export", 
        params={"page": page, "per_page": 10, "brand": "Volkswagen"},
        auth=HTTPBasicAuth(USERNAME, PASSWORD),
        cert=(crt_file, pem_file)
    )

    # Dynamically determine the number of items on the current page
    data = resp.json().get('data', [])
    for i, dealer in enumerate(data):
        try:
            conc = dealer['address'][0]
            if 'longitude' in conc and 'latitude' in conc:
                d = {k: conc[k] for k in ['line','city', 'country','postalCode', 'longitude', 'latitude']}
                concessions_exploitables.append(d)
            else:
                d = {k: conc[k] for k in ['line','city','country', 'postalCode']}
                concessions_non_exploitables.append(d)
        except (IndexError, KeyError) as e:
            print(f"Error at page {page}, item {i}: {e}")

In [28]:
# page = 1
# per_page = 10

# while True:
#     try:
#         resp = requests.get(
#             "https://api.non-prod.volkswagengroup.fr/tools/qa/leads/v2/dealers/export", 
#             params={"page": page, "per_page": per_page, "brand": "Volkswagen"},
#             auth=HTTPBasicAuth(USERNAME, PASSWORD),
#             cert=(crt_file, pem_file)
#         )
#         # Vérification du statut de la réponse
#         if resp.status_code != 200:
#             print(f"Reached end of pages at page {page-1}.")
#             break
        
#         data = resp.json()  # Récupérer les données au format JSON
#         if not data['data']:
#             print(f"No data found on page {page}. Stopping.")
#             break
        
#         # Traite les données de cette page ici
#         print(f"Processing page {page}: {data}")

#         page += 1
    
#     except requests.exceptions.RequestException as e:
#         print(f"Error occurred: {e}")
#         break

Processing page 1: {'data': [{'identifier': [{'use': 'business', 'type': 'kvps', 'value': '05688'}, {'use': 'business', 'type': 'kvps_long', 'value': 'FRAV05688'}, {'use': 'system', 'type': 'sourceId', 'value': '001bg000001NbhbAAC'}], 'name': 'JEAN LAIN AUTOMOBILES MAURIENNE', 'brand': 'Volkswagen', 'startDate': '2024-06-20T14:32:46.000Z', 'leadManagementTool': 'UpYourLeads', 'contract': [{'id': '800bg000000o5KWAAY', 'name': 'Distributeur VW', 'number': '01005688', 'activityType': 'Distributeur VW', 'brand': 'Volkswagen', 'bdmName': 'Arnaud-PASQUIER', 'status': 'Active', 'bevAgency': 'false', 'activity': [{'id': 'a4Dbg0000000QkpEAE', 'name': 'CRD Réseau', 'contractNumber': '01005688', 'status': 'true'}, {'id': 'a4Dbg0000000QkqEAE', 'name': 'Commercial', 'contractNumber': '01005688', 'status': 'true'}, {'id': 'a4Dbg0000000QkrEAE', 'name': 'Fleet Agency', 'contractNumber': '01005688', 'status': 'true'}, {'id': 'a4Dbg0000000QksEAE', 'name': 'Formation', 'contractNumber': '01005688', 'stat

In [104]:
concessions_exploitables

[{'city': 'ST JEAN DE MAURIENNE',
  'postalCode': '73300',
  'longitude': '6.36427778',
  'latitude': '45.27122778'},
 {'city': 'ST MAXIMIN LA STE BAUME',
  'postalCode': '83470',
  'longitude': '5.84792778',
  'latitude': '43.45526389'},
 {'city': 'PONT DE CHERUY',
  'postalCode': '38230',
  'longitude': '5.84792778',
  'latitude': '43.45526389'},
 {'city': 'VARENNES SUR SEINE',
  'postalCode': '77130',
  'longitude': '2.95686667',
  'latitude': '48.36768333'},
 {'city': 'COLMAR',
  'postalCode': '68000',
  'longitude': '7.37905278',
  'latitude': '48.10846111'},
 {'city': 'TRIGNAC',
  'postalCode': '44570',
  'longitude': '-2.184379',
  'latitude': '47.323167'},
 {'city': 'CESSON SEVIGNE CEDEX',
  'postalCode': '35511',
  'longitude': '-1.6169826',
  'latitude': '48.1054341'},
 {'city': 'STE GENEVIEVE BOIS CEDEX',
  'postalCode': '91704',
  'longitude': '2.33358056',
  'latitude': '48.64096389'},
 {'city': 'TOULOUSE',
  'postalCode': '31100',
  'longitude': '2.17788611',
  'latitude'

In [125]:
from geopy.geocoders import Nominatim
import certifi
import ssl 

def get_lat_lon(zip_code, country="France"):
    ssl_context = ssl.create_default_context(cafile=certifi.where())
    geolocator = Nominatim(user_agent="geoapi", ssl_context=ssl_context)
    location = geolocator.geocode(f"{zip_code}, {country}")
    
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

# # Example usage
# zip_code = "75001"  # ZIP code for Paris, France
# lat_lon = get_lat_lon(zip_code=zip_code)

In [119]:
float(concessions[0]['latitude'])

45.27122778

In [126]:
client_coords = get_lat_lon(78000)
concessions = concessions_exploitables

def find_nearest_concession(client_zip_code, concessions):
    if client_coords is None:
        return "Invalid ZIP code or location not found."

    # Extract concession coordinates (latitude, longitude)
    concession_coords = np.array([[float(conc['latitude']), float(conc['longitude'])] for conc in concessions])
    # Step 3: Use KNN to find the nearest concession
    knn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')  # You can also use 'auto', 'kd_tree', or 'brute'
    knn.fit(concession_coords)

    # Find the nearest concession to the client's coordinates
    distance, index = knn.kneighbors([client_coords])

    # Return the nearest concession
    nearest_concession = concessions[index[0][0]]
    return (nearest_concession, distance[0][0])

{'line': [{'use': 'street', 'value': '10 Rue DE LA CELLE  LE CHESNAY'}], 'city': 'LE CHESNAY', 'country': 'France', 'postalCode': '78150', 'longitude': '2.12591389', 'latitude': '48.81850278'} 0.016151105073428113
